In [118]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import numpy as np
from tqdm import tqdm

movies = pd.read_excel('movie15.xls', skiprows=4)
movies.head()

,영화명,영화명(영문),제작연도,제작국가,유형,장르,제작상태,감독,제작사
0,부활의 노래,The Song Of Resurrection,1990.0,한국,장편,드라마,개봉,이정국,새빛영화제작소
1,젖은장모-사위 대물에 눈 뒤집히네,NaN,2022.0,일본,장편,"성인물(에로),드라마,멜로/로맨스",개봉,츠지 코지,NaN
2,점장님 팬티 내릴게요 눈감아 주세요,NaN,2022.0,일본,장편,"성인물(에로),드라마,멜로/로맨스",개봉,타카야마 켄타로,NaN
3,옆집여자-남편 대신 시원하게 싸줬다,NaN,2023.0,일본,장편,"성인물(에로),드라마,멜로/로맨스",개봉,타카야마 켄타로,NaN
4,물 넘치는 섹파-방금 쌌는데 또 싸고싶어,NaN,2023.0,일본,장편,"성인물(에로),드라마,멜로/로맨스",개봉,히무 롯쿠,NaN


In [119]:
def genre(x):
    if '성인물(에로)' not in x:
        return x
movies['장르'] = movies.장르.apply(genre)
movies = movies[~movies['장르'].isnull()].reset_index(drop=True)
print(len(movies.장르.unique()))
movies.head()

885


,영화명,영화명(영문),제작연도,제작국가,유형,장르,제작상태,감독,제작사
0,부활의 노래,The Song Of Resurrection,1990.0,한국,장편,드라마,개봉,이정국,새빛영화제작소
1,이매큘레이트,Immaculate,2024.0,미국,장편,공포(호러),개봉,마이클 모한,NaN
2,러브 라이즈 블리딩,Love Lies Bleeding,2023.0,미국,장편,"범죄,멜로/로맨스",개봉,로즈 글래스,NaN
3,짱,Top Dog,2018.0,한국,장편,액션,개봉,송재덕,"(주)영화사 사계절,펀펀한영화사"
4,싸움의 기술 2,The Art of Fighting 2,2020.0,한국,장편,액션,개봉,김희성,(주)영화사 사계절


In [120]:
movies = movies.영화명.tolist()
movies[:15]

['부활의 노래',
 '이매큘레이트',
 '러브 라이즈 블리딩',
 '짱',
 '싸움의 기술 2',
 '아이윌 송',
 '싸움의 기술',
 '깡패들',
 '전야',
 '심장박동 조작극',
 '비포 선셋',
 '사일런트 나잇',
 '명탐정 코난: 100만 달러의 펜타그램',
 '그리고 바통은 넘겨졌다',
 '더 납작 엎드릴게요']

In [150]:
url = 'https://namu.wiki/w/' # %EC%82%B4%EC%9D%B8%EC%9D%98%20%EC%B6%94%EC%96%B5
pd_movies = []
content = ''
cnt = 0

# page = requests.get(url + '깡패들') # 탈주(영화)
page = requests.get(url + '빅샤크5: 80일간의 해저일주') 
html = BeautifulSoup(page.text, 'html.parser')

soup1 = html.select('#app div div div div') # 시놉시스
soup2 = html.select('#app div div div div .wiki-paragraph') # 테이블 형식 데이터 수집

In [151]:
evaluation = []
st = 0

for s in soup1:
    if s.text != '':
        if '시놉시스[편집]' in s.text:
            content = s.text
            break
lst = content.split('[편집]')
for i, v in enumerate(lst):
    if v[-4:] == '시놉시스':
        synopsys = lst[i+1]
        break
for i, v in enumerate(soup1):
    if '평가[편집]' in v.text:
        evaluation.append(soup1[i:i+10])

In [152]:
synopsys

'뒤죽박죽 위험에 처한 바다왕국을 구해라! 태평양을 지나 북극해와남극해까지... 80일간의 짜릿한 깊은 바닷속 해저일주 모험이 펼쳐진다.평화로운 바다 왕국. 어느 날 해저지진과 함께 심해의 해양생물들이감쪽같이 사라지고, 다른 차원의 문이 열리기 시작한다. 80일 후면바다 왕국 전체가 사라질 긴박한 상황. 빅샤크와 해저 탐험대 친구들은해저몬스터의 음모를 막고 뒤죽박죽 위험에 빠진 바다를 구하기 위해태평양을 거쳐 대서양, 인도양, 그리고 북극해와 남극해까지 80일간의스펙터클하고 짜릿한  해저 일주 모험을 시작한다.4. 예고편'

In [145]:
evaluation = evaluation[-1]
lst = [] # 영화에 대한 정보를 담는 리스트
for s in soup2[1:]: # 영화의 정보, 1은 총 데이터 묶음이므로 생략
    if s.text != '':
        if s.text == '장르':
            st = 1
        # if '장르' in s.text:
        #     st = 1
        elif '개요' in s.text:
            st = 0
            break
        if st == 1:
            lst.append(s.text) # 영화 정보 담기
    if '개요' in s.text:
        break

table_data = []
data = []
idx = -1

In [146]:
lst

['장르',
 '액션, 스릴러, 밀리터리, 드라마',
 '감독',
 '이종필',
 '각본',
 '권성휘[1], 김우근',
 '각색',
 '이종필, 김민석, 문병곤',
 '제작',
 '박은경, 권성휘, 손다올',
 '제작투자',
 '홍정인',
 '프로듀서',
 '강대욱, 김우성',
 '촬영',
 '김성안',
 '조명',
 '이승빈',
 '미술',
 '배정윤',
 '분장',
 '김현정',
 '편집',
 '이강희',
 '음악',
 '달파란',
 '시각효과',
 'RASCA',
 '출연',
 '이제훈, 구교환, 홍사빈 外',
 '제작사',
 '더램프, 은하수필름[2]',
 '배급사',
 ' 플러스엠',
 '개봉일',
 ' 2024년 7월 3일',
 '촬영 기간',
 '2022년 2월 20일 ~ 2022년 6월 7일',
 '화면비',
 '2.39:1',
 '상영 타입',
 '2D, Dolby Atmos',
 '상영 시간',
 '94분 (1시간 34분 9초)',
 '제작비',
 '85억 원[3]',
 '대한민국 총 관객 수',
 '1,957,787명 (2024년 7월 22일 기준)',
 '상영 등급',
 ' 12세 이상 관람가[4]']

In [148]:
if len(evaluation[idx]) == 0: # 페이지마다 스크래핑되는 경우와 안되는 경우가 있음.
    idx = -2                  # 인덱스를 바꾸면 가져올 수 있음

for i in evaluation[idx].select('a'):
    if ('http' not in i.attrs['title']) & (i.attrs['title'] != '') & (len(i.attrs['title']) < 30):
        try:
            if '네이버 영화' in table_data[-1]: # 네이버 영화는 동일한 데이터가 2번 입력됨
                del data[0]
        except:
            pass
        table_data.append(data)
        table_data.append(i.attrs['title'])
        data = []
    if (i.text != '') & bool(re.search(r'[\d%/]', i.text)) & (len(data) < 2) & ~bool(re.search(r'[\[\]]', i.text)): 
        # 데이터값에는 다양한 데이터가 들어가지만 주로 1/10 같은 분수 표현과 숫자로 구성 되어 있음. 
        # 또한 %도 들어가며 데이터는 최대 2개만 들어가는 것으로 보임
        data.append(i.text)

In [149]:
table_data

[]

In [60]:
# 평가 기관 및 점수 데이터 가공
table_data = table_data[1:] # 첫째 데이터는 비어있음
eval_dict = dict(zip(table_data[::2],table_data[1::2])) # dict화

max_length = max(len(v) for v in eval_dict.values()) # 평가 챕터가 열려있지 않으면 에러 발생하므로 try문으로 묶음
for k, v in eval_dict.items(): # 데이터 길이 맞추기
    if len(v) < max_length:
        eval_dict[k] = v + [np.nan] * (max_length - len(v))

eval_df = pd.DataFrame(eval_dict) # 데이터프레임화
eval_df = eval_df.dropna(axis=1, how='all') # 데이터가 전부 비어있으면 삭제
eval_df = eval_df.transpose().reset_index()
eval_df.columns = ['평가', '점수1', '점수2']

# 영화 정보 가공
lst.insert(0, '영화명')
lst.insert(1, '러브 라이즈 블리딩')
df = pd.DataFrame(lst)
df = pd.concat([df.iloc[::2].reset_index(drop=True), df.iloc[1::2].reset_index(drop=True)], axis=1)
df.columns = ['kind', 'content']
df.set_index(df.kind, inplace=True)
df = df.drop('kind', axis=1)
pd_movies.append([eval_df,df])

In [65]:
table_data

['IMDb',
 [' 평점 7.1 / 10'],
 '로튼 토마토',
 [' 신선도 개봉전%', ' 관객 점수 개봉전%'],
 '메타크리틱',
 [' 메타스코어  / 100', ' 점수  / 10'],
 '알로시네',
 [' 언론 별점 3.0 / 5.0', ' 관객 별점 3.4 / 5.0'],
 'Letterboxd',
 [' 별점 3.4 / 5.0'],
 '키노포이스크',
 [' 별점 개봉전 / 10'],
 '야후! 재팬',
 [' 별점  / 5.0'],
 'Filmarks',
 [' 별점 3.9 / 5.0'],
 '더우반',
 [' 별점 6.8 / 10', ' 별점  / 10'],
 '왓챠피디아',
 [' 별점 3.2 / 5.0'],
 '키노라이츠',
 [' 지수 97.3%', ' 별점 3.2 / 5.0'],
 '네이버',
 [' 평점  / 10'],
 'CGV Golden EGG지수',
 [' 지수 94%'],
 '롯데시네마',
 [' 평점 개봉전 / 10'],
 '메가박스',
 [' 평점 개봉전 / 10'],
 'MRQE',
 [' 평점  / 100'],
 '시네마스코어',
 [],
 'TMDB',
 [' 점수 75%'],
 '무비파일럿',
 [' 평점 개봉전 / 10'],
 '로저 이버트 닷컴',
 [' 별점  / 4'],
 'MyDramaList',
 [' 별점  / 10'],
 'Laftel',
 [' 별점  / 5.0'],
 'MyAnimeList',
 [' 평점 7.68 / 10.0'],
 'Google Play 무비',
 [' 별점  / 5.0'],
 '네이버 시리즈온',
 [' 별점  / 10'],
 'JustWatch',
 [' 평점 %'],
 '맥스무비',
 [' 평점  / 100'],
 '씨네21',
 [' 씨네21 XX / 10', ' 별점 XX / 10'],
 'CGV/해외사업',
 [' 평점  / 10'],
 '엠파이어',
 [' 별점  / 5'],
 'IMDb',
 [' 평점 7.1 / 10'],
 '로튼 토마토',
 [' 신선

In [63]:
pd_movies[0][1]

,content
kind,
영화명,러브 라이즈 블리딩
